<a href="https://colab.research.google.com/github/Sdmr12012003/Neural_Networks_and_Large_Language_Models/blob/main/Generative_AI__Using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]

In [ ]:
pip install torch

In [ ]:
!pip install torchdata

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("Salesforce/codet5-small")

In [ ]:
example = ["Do not meddle in the affairs of wizards,","for they are subtle and quick to anger."]
encoded_input = tokenizer(example,padding = True, return_tensors = 'pt')
print(encoded_input)

In [ ]:
encoded_input1 = tokenizer1(example,padding = True, return_tensors = 'pt')
print(encoded_input1)

In [ ]:
from transformers import AutoConfig
details = AutoConfig.from_pretrained("google/flan-t5-base")
details

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained('google/flan-t5-base')
model

In [ ]:
okk = AutoConfig.from_pretrained('google/flan-t5-base')
okk1 = AutoModel.from_config(okk)
okk1

In [ ]:
from transformers import AutoModelForPreTraining
pre_okk = AutoModelForPreTraining.from_config(okk)
pre_okk

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

data = load_dataset('knkarthick/dialogsum')
data

In [ ]:
print(data['train'][100]['dialogue'])

In [ ]:
from transformers import AutoModelForSeq2SeqLM
models = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')
models

In [ ]:
decoded_input = tokenizer1.decode(encoded_input1['input_ids'][1], skip_special_tokens = True)
decoded_input

In [ ]:
d = [40,200]
for i,index in enumerate(d):
  dialogue = data['test'][index]['dialogue']
  summary = data['test'][index]['summary']

  inputs = tokenizer(dialogue, return_tensors = 'pt')
  output = tokenizer.decode(models.generate(inputs['input_ids'], max_new_tokens = 50)[0], skip_special_tokens = True)

  print(summary)
  print(output)
  print('\n')

In [ ]:
for i,index in enumerate(d):
  dialogue = data['test'][index]['dialogue']
  summary = data['test'][index]['summary']

  prompt = f"""

  Summarize the following conversation:
  {dialogue}

  Summary:

   """

  inputs = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(models.generate(inputs['input_ids'], max_new_tokens = 50)[0], skip_special_tokens = True)

  print(summary)
  print(output)
  print('\n')

In [ ]:
for i,index in enumerate(d):
  dialogue = data['test'][index]['dialogue']
  summary = data['test'][index]['summary']

  prompt = f"""

  Summarize the following conversation:
  {dialogue}

  What is going on?

   """

  inputs = tokenizer(prompt, return_tensors = 'pt')
  output = tokenizer.decode(models.generate(inputs['input_ids'], max_new_tokens = 50)[0], skip_special_tokens = True)

  print(summary)
  print(output)
  print('\n')

In [ ]:
def prompt(example_full_index,example_summarize_index):
  prompt = ''
  for i,index in enumerate(example_full_index):
    dialogue = data['test'][index]['dialogue']
    summary = data['test'][index]['summary']

    prompt += f"""

    Summarize the following conversation:
    {dialogue}

    What is going on?
    {summary}

    """
  dialogue1 = data['test'][example_summarize_index]['dialogue']

  prompt += f"""

  Summarize the following conversation:
  {dialogue1}

  What is going on?
  """

  return prompt

In [ ]:
example_full_index = [40]
example_summarize_index = 200

one_shot_prompt = prompt(example_full_index,example_summarize_index)
one_shot_prompt

In [ ]:
input = tokenizer(one_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , max_new_tokens = 50)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
example_full_index = [40,80,100]
example_summarize_index = 200

few_shot_prompt = prompt(example_full_index,example_summarize_index)
few_shot_prompt

In [ ]:
input = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , max_new_tokens = 50)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(max_new_tokens = 50)
generation_config1 = GenerationConfig(max_new_tokens = 50, do_sample = True , temperature = 0.1)
generation_config2 = GenerationConfig(max_new_tokens = 50, do_sample = True , temperature = 0.5)
generation_config3 = GenerationConfig(max_new_tokens = 50, do_sample = True , temperature = 1.0)

input = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , generation_config = generation_config)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
input = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , generation_config = generation_config1)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
input = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , generation_config = generation_config2)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
input = tokenizer(few_shot_prompt, return_tensors = 'pt')
output = tokenizer.decode(models.generate(input['input_ids'] , generation_config = generation_config3)[0], skip_special_tokens = True)
summary = data['test'][example_summarize_index]['summary']
print(summary)
print(output)

In [ ]:
pip install rouge_score

In [ ]:
pip install evaluate

In [ ]:
pip install peft

In [ ]:
pip install loralib

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import pandas as pd
import numpy as np
import time

In [ ]:
#lets print number of trainable parameter

def print_number_of_trainable_parameters(model):
  trainable_parameters = 0
  all_parameters = 0
  for _,param in models.named_parameters():
    all_parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  print(f"Number of Trainable Parameters : {trainable_parameters} \nNumber of All Parameters : {all_parameters}")

In [ ]:
print_number_of_trainable_parameters(models)

In [ ]:
!nvidia-smi

In [ ]:
import os
from datetime import datetime
import argparse
import torch.multiprocessing as mp
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.distributed as dist


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
models = models.to(device)

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation:'
    end_prompt = '\n\nSummary:'

    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = torch.tensor(tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt').input_ids).to(device).long()
    example['label_ids'] = torch.tensor(tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt').input_ids).to(device).long()

    print(example['input_ids'].dtype)
    return example

tokenized_dataset = data.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['id','topic','dialogue','summary'])


In [ ]:
pd.DataFrame(tokenized_dataset['train'])

In [ ]:
tokenized_dataset

In [ ]:
pd.DataFrame(tokenized_dataset['train'])

In [ ]:
# Example to check the format of input_ids in the training dataset
training_sample = tokenized_dataset['train'][0]  # Get the first sample from the training set

# Access and print the input_ids from the sample
input_ids_sample = training_sample['input_ids']
print(f"Data Type of input_ids: {type(input_ids_sample)}")

# Check the length of the list (number of tokens in input_ids)
print(f"Length of input_ids list: {len(input_ids_sample)}")

# Print an example token sequence from the input_ids list
print(f"Example input_ids: {input_ids_sample}")



In [ ]:
pip install accelerate>=0.20.1

In [ ]:
print(f"Shape of Dataset:")
print(f"Shape of train dataset : {tokenized_dataset['train'].shape}")
print(f"Shape of train dataset : {tokenized_dataset['validation'].shape}")
print(f"Shape of train dataset : {tokenized_dataset['test'].shape}")

print(tokenized_dataset)

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs = 1,
    weight_decay = 0.01,
    logging_steps = 1,
    max_steps = 1,
    per_device_train_batch_size=2,  # Decrease batch size
    per_device_eval_batch_size=2,   # Decrease eval batch size if necessary
    gradient_accumulation_steps=4  # Accumulate gradients across 4 step
)

trainer = Trainer(
    model = models,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation']
)

In [ ]:
trainer.train()

In [ ]:
instruct_model_paths = './instruct_model'

trainer.save_model(instruct_model_paths)

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_paths)
instruct_model

In [ ]:
orignal_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

In [ ]:
index = 200

dialogue = data['test'][index]['dialogue']
human_summary = data['test'][index]['summary']

prompt = f"""
Summarize the following :
{dialogue}

Summary:
 """

input_ids = tokenizer(prompt, truncation = True , return_tensors = 'pt').input_ids
orignal_model_output = orignal_model.generate(input_ids, generation_config = GenerationConfig(max_new_tokens = 50,do_sample = True , temperature = 0.1))
orignal_model_text = tokenizer.decode(orignal_model_output[0] , skip_special_tokens = True)

instruct_model_output = instruct_model.generate(input_ids, generation_config = GenerationConfig(max_new_tokens = 50,do_sample = True , temperature = 0.1))
instruct_model_text = tokenizer.decode(instruct_model_output[0] , skip_special_tokens = True)

dash_line = "--------------------------------------"
print(dash_line)
print('BASLINE HUMAN SUMMARY\n')
print(human_summary)
print(dash_line)
print('ORIGNAL_MODEL SUMMARY\n')
print(orignal_model_text)
print(dash_line)
print('ISTRUCT_FULL_FINE_TUNED MODEL SUMMARY')
print(instruct_model_text)
